In [ ]:
from load_paus_mocks import load_mock_dict, add_errors
from LAE_selection_method import select_LAEs
from paus_utils import w_central
from jpasLAEs.utils import flux_to_mag

import numpy as np

import matplotlib
import matplotlib.pyplot as plt
matplotlib.rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
matplotlib.rc('text', usetex=True)
matplotlib.rcParams.update({'font.size': 16})

In [ ]:
gal_fraction = 0.2

source_cats_dir = '/home/alberto/almacen/Source_cats'
mock_SFG_path = f'{source_cats_dir}/LAE_12.5deg_z2.55-5_PAUS_0'
mock_QSO_cont_path = f'{source_cats_dir}/QSO_PAUS_contaminants_2'
mock_QSO_LAEs_loL_path = f'{source_cats_dir}/QSO_PAUS_LAES_2'
mock_QSO_LAEs_hiL_path = f'{source_cats_dir}/QSO_PAUS_LAES_hiL_2'
mock_GAL_path = '/home/alberto/almacen/PAUS_data/catalogs/LightCone_mock.fits'
mock_dict = load_mock_dict(mock_SFG_path, mock_QSO_cont_path,
                           mock_QSO_LAEs_loL_path, mock_QSO_LAEs_hiL_path,
                           mock_GAL_path, gal_fraction=gal_fraction)

# State the mock area in deg²:
area_dict = {
    'SFG': 400,
    'QSO_cont': 200,
    'QSO_LAEs_loL': 400,
    'QSO_LAEs_hiL': 4000,
    'GAL': 59.97 * gal_fraction
}

for mock_name, area in area_dict.items():
    mock_dict[mock_name]['area'] = area

# Add errors
field_name = 'W3'
for _, mock in mock_dict.items():
    mock['flx'], mock['err'] = add_errors(mock['flx_0'], field_name,
                                          add_errors=True)
    # Compute r_mag
    mock['r_mag'] = flux_to_mag(mock['flx'][-4], w_central[-4])

In [ ]:
ew0_min_list = [0, 10, 20, 30, 40]

nb_min, nb_max = 0, 10
r_min, r_max = 17, 24

sel_dict = {}

for ew0_min in ew0_min_list:
    sel_dict[f'EW0_min_{ew0_min}'] = {}

    for mock_name, mock in mock_dict.items():
        mock = select_LAEs(mock, nb_min, nb_max, r_min, r_max,
                        ew0_min, ewmin_other=100, check_nice_z=True)

        sel_dict[f'EW0_min_{ew0_min}'][mock_name] = {}

        sel_dict[f'EW0_min_{ew0_min}'][mock_name]['nice_lya'] = mock['nice_lya']
        sel_dict[f'EW0_min_{ew0_min}'][mock_name]['nice_z'] = mock['nice_z']
        sel_dict[f'EW0_min_{ew0_min}'][mock_name]['L_lya'] = mock['L_lya']

In [ ]:
import pickle
with open('/home/alberto/Desktop/sel_dict.pkl', 'wb') as f:
    pickle.dump(sel_dict, f)

In [ ]:
from jpasLAEs.utils import bin_centers
from paus_utils import z_NB

L_bins = np.linspace(42, 45.5, 30)
L_bins_c = bin_centers(L_bins)

h_parent = np.zeros(len(L_bins) - 1)

z_min, z_max = z_NB(nb_min), z_NB(nb_max)

for mock_name, mock in mock_dict.items():
    zspec = mock['zspec']
    zspec_mask = (zspec >= z_min) & (zspec <= z_max)
    h_parent += np.histogram(mock['L_lya_spec'][zspec_mask], L_bins)[0] / area_dict[mock_name]

In [ ]:
fig, axs = plt.subplots(2, figsize=(6, 8), sharex=True)

for ew0_min in ew0_min_list:
    h_sel = np.zeros(len(L_bins) - 1)
    h_nice = np.zeros(len(L_bins) - 1)
    h_nice_spec = np.zeros(len(L_bins) - 1)

    for mock_name, mock in mock_dict.items():
        nice_lya = sel_dict[f'EW0_min_{ew0_min}'][mock_name]['nice_lya']
        nice_z = sel_dict[f'EW0_min_{ew0_min}'][mock_name]['nice_z']
        L_lya = sel_dict[f'EW0_min_{ew0_min}'][mock_name]['L_lya']
        zspec = mock['zspec']
        zspec_mask = (zspec >= z_min) & (zspec <= z_max)

        h_sel += np.histogram(L_lya[nice_lya], L_bins)[0] / area_dict[mock_name]
        h_nice += np.histogram(L_lya[nice_lya & nice_z & zspec_mask], L_bins)[0] / area_dict[mock_name]
        h_nice_spec += np.histogram(mock['L_lya_spec'][nice_lya & nice_z & zspec_mask], L_bins)[0] / area_dict[mock_name]

    puri1d = h_nice / h_sel
    comp1d = h_nice_spec / h_parent

    puri1d[~np.isfinite(puri1d)] = 0.

    axs[0].plot(L_bins_c, puri1d, label=f'{ew0_min=}')
    axs[1].plot(L_bins_c, comp1d)

axs[0].set_ylabel('Purity')
axs[1].set_ylabel('"Completeness"')

axs[0].legend(fontsize=11)

# for ax in axs:
#     ax.set_ylim(0, 1)

plt.show()